In [57]:
import torch
import cv2
import numpy as np

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [59]:
def create_gradient_mask(height, width, padding_vertical, padding_horizontal):
    mask = np.ones((height, width), dtype=np.float32)
    
    if padding_vertical > 0:
        for i in range(padding_vertical):
            grad_value = (i / padding_vertical) ** 2
            mask[i, :] = grad_value
            mask[height - 1 - i, :] = grad_value

    if padding_horizontal > 0:
        for i in range(padding_horizontal):
            grad_value = (i / padding_horizontal) ** 2
            mask[:, i] = grad_value
            mask[:, width - 1 - i] = grad_value

    return mask

def add_blurred_padding(image_path, blur_strength=25, sigma=0):
    img = cv2.imread(image_path)

    height, width, channels = img.shape

    if height > width:
        padding_vertical = 0
        padding_horizontal = (height - width) // 2
    else:
        padding_vertical = (width - height) // 2
        padding_horizontal = 0

    padded_img = cv2.copyMakeBorder(
        img, 
        padding_vertical, padding_vertical, padding_horizontal, padding_horizontal, 
        borderType=cv2.BORDER_REFLECT
    )

    blurred_img = cv2.GaussianBlur(padded_img, (blur_strength, blur_strength), sigma)

    mask = create_gradient_mask(padded_img.shape[0], padded_img.shape[1], padding_vertical, padding_horizontal)

    mask_3d = np.repeat(mask[:, :, np.newaxis], 3, axis=2)

    blended_img = (mask_3d * padded_img + (1 - mask_3d) * blurred_img).astype(np.uint8)

    return blended_img


def show_image(image):
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    cv2.imshow("Padded Image", img_rgb)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [60]:
image_path = "Slag & Shore.jpg"
padded_image = add_blurred_padding(image_path, 25, 0)
cv2.imwrite("padded_output.jpg", padded_image)
show_image(padded_image)